In [5]:
import os, sys
import numpy as np
import xarray as xr
import pandas as pd

In [6]:
filename =  '/gnn/rrr/integrated_weather_dataset/data/raw/Guan_AR_Catalog/globalARcatalog_MERRA2_1980-2023_v4.0.nc'
ds = xr.open_dataset(filename, chunks={'time': 1460}, engine='netcdf4')
ds = ds.squeeze()
ds = ds.reset_coords(names=['lev', 'ens'], drop=True)

/tmp/ipykernel_2788/2619948906.py:2: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1460. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(filename, chunks={'time': 1460}, engine='netcdf4')


In [7]:
# Step 2: Select the relevant geographical region
MIN_LON = -120+360
MAX_LON = -115+360
MIN_LAT = 31.5
MAX_LAT = 38
start_date = '2017-01-01 00:00'
end_date = '2017-02-01 00:00'
ds = ds.sel(time=slice(start_date, end_date), lat=slice(MIN_LAT,MAX_LAT), lon=slice(MIN_LON,MAX_LON))
print(f"Data selected for lat: {MIN_LAT} to {MAX_LAT} and lon: {MIN_LON} to {MAX_LON}")


Data selected for lat: 31.5 to 38 and lon: 240 to 245


In [8]:
ds

<xarray.Dataset> Size: 1MB
Dimensions:      (lon: 9, lat: 14, time: 125)
Coordinates:
  * lon          (lon) float64 72B 240.0 240.6 241.2 241.9 ... 243.8 244.4 245.0
  * lat          (lat) float64 112B 31.5 32.0 32.5 33.0 ... 36.5 37.0 37.5 38.0
  * time         (time) datetime64[ns] 1kB 2017-01-01 ... 2017-02-01
Data variables: (12/55)
    islnd        (lat, lon) float32 504B dask.array<chunksize=(14, 9), meta=np.ndarray>
    iscst        (lat, lon) float32 504B dask.array<chunksize=(14, 9), meta=np.ndarray>
    ivt_begin    (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
    ivt_end      (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
    shapemap     (time, lat, lon) float32 63kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    axismap      (time, lat, lon) float64 126kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    ...           ...
    kinstveldir  (time, lat) float32 7kB dask.array<chunksize=(125, 14), meta=np.ndarray>
    axisidx      (time, lat, lon) float32 63kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    axislon      (time, lat, lon) float32 63kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    axislat      (time, lat, lon) float32 63kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    numobj       (time, lat, lon) float32 63kB dask.array<chunksize=(125, 14, 9), meta=np.ndarray>
    intblosto    (lat, lon) float64 1kB dask.array<chunksize=(14, 9), meta=np.ndarray>
Attributes:
    title:                 Global Atmospheric River Database
    version:               2024.05.17
    creation_date:         17-May-2024 11:53:44
    tARget_version:        1.312
    dimension_convention:  5-d (nlon x nlat x nlev x ntime x nens) variables ...
    references:            (1) Guan, B., and D. E. Waliser (2015), Detection ...

In [9]:
df = ds.kidmap.to_dataframe(dim_order=['time', 'lat', 'lon']).dropna()
print(df)
print(df.info())
print(df.head())


                                        kidmap
time                lat  lon                  
2017-01-02 12:00:00 31.5 240.625  2.016123e+11
                         241.250  2.016123e+11
                         241.875  2.016123e+11
2017-01-03 00:00:00 31.5 240.625  2.017010e+11
                         241.250  2.017010e+11
...                                        ...
2017-01-24 00:00:00 33.5 243.750  2.017012e+11
                         244.375  2.017012e+11
                         245.000  2.017012e+11
                    34.0 245.000  2.017012e+11
                    34.5 245.000  2.017012e+11

[4762 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4762 entries, (Timestamp('2017-01-02 12:00:00'), np.float64(31.5), np.float64(240.625)) to (Timestamp('2017-01-24 00:00:00'), np.float64(34.5), np.float64(245.0))
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kidmap  4762 non-null   float64
dtypes: 

In [17]:
df = df.reset_index()

In [18]:
df['Label'] = df['kidmap'].apply(lambda x: 1 if x > 0 else 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4762 entries, 0 to 4761
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    4762 non-null   datetime64[ns]
 1   lat     4762 non-null   float64       
 2   lon     4762 non-null   float64       
 3   kidmap  4762 non-null   float64       
 4   Label   4762 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 186.1 KB


In [19]:
print(df['lat'].unique())  # Example of accessing unique latitude values


[31.5 32.  32.5 33.  33.5 34.  34.5 35.  35.5 36.  36.5 37.  37.5 38. ]


In [12]:
all_days = pd.date_range(start=start_date, end=end_date, freq='1D')

# Step 4: Create all possible combinations of days, latitudes, and longitudes
all_combinations = pd.MultiIndex.from_product(
        [all_days, df['lat'].unique(), df['lon'].unique()],
        names=['time', 'lat', 'lon']
    ).to_frame(index=False)


KeyError: 'lat'